In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
emotion = pd.read_csv('emotion_dataset_balanced.csv')

In [3]:
emotion.head()

,Unnamed: 0,Emotion,Text,Clean_Text
0,14929,anger,"@charlottehanlan i agree, angry birds is exact...",agree angry birds exactly described
1,21962,anger,CRAP! Didn't think I'd have to know and unders...,CRAP think Id know understand statistics again...
2,15735,anger,Don ’ t criticize Mario or else I ’ ll start ...,’ criticize Mario ’ start fat drunk friends ...
3,18326,anger,Are you done ?,NaN
4,29278,anger,He was coming so fast that she knew he had not...,coming fast knew seen brake sharply frowning ...


In [4]:
print(emotion.shape)

(13524, 4)


In [5]:
print('Emotions are: ', emotion['Emotion'].unique())

Emotions are:  ['anger' 'fear' 'joy' 'neutral' 'sadness' 'surprise']


In [6]:
emotion.isnull().sum()

Unnamed: 0      0
Emotion         0
Text            0
Clean_Text    452
dtype: int64

In [7]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
emotion['Emotion'].value_counts()

Emotion
anger       2254
fear        2254
joy         2254
neutral     2254
sadness     2254
surprise    2254
Name: count, dtype: int64

In [9]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import spacy

from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shamimkhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
emotion['Clean_Text'] = emotion['Clean_Text'].fillna('')

In [11]:
def remove_html(text):
  bs = BeautifulSoup(text, 'html.parser')
  return ' ' + bs.get_text() + ' '

def keep_only_letters(text):
  text = re.sub(r'[^a-zA-Z\s]', ' ', text)
  return text

def convert_to_lowercase(text):
  return text.lower()

def clean_reviews(text):
  text = remove_html(text)
  text = keep_only_letters(text)
  text = convert_to_lowercase(text)
  return text


emotion['Clean_Text'] = emotion['Clean_Text'].apply(clean_reviews)

In [12]:
english_stop_words = nltk.corpus.stopwords.words('english')

print(f"Total number of English stopwords: {len(english_stop_words)}")
print("\nFirst 20 stopwords:")
print(english_stop_words[:20])

Total number of English stopwords: 198

First 20 stopwords:
['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been']


In [19]:
def remove_stop_words(text):
  for stopword in english_stop_words:
    stopword = ' ' + stopword + ' '
    text = text.replace(stopword, ' ')
    return text

emotion['Clean_Text'] = emotion['Clean_Text'].apply(remove_stop_words)

In [20]:
def text_stemming(text):
  stemmer = nltk.porter.PorterStemmer()
  stemmed = ' '.join([stemmer.stem(token) for token in text.split()])
  return stemmed

emotion['Text'] = emotion['Text'].apply(text_stemming)

In [21]:
emotion = emotion.sample(frac=1).reset_index(drop=True)

In [22]:
from sklearn.model_selection import train_test_split

# 1. Define your features (X) and target (y)
# X will be the 'Clean_Text' column you processed
# y will be the 'Emotion' column
X = emotion['Clean_Text']
y = emotion['Emotion']

# 2. Split the data into training and testing sets
# We use test_size=0.2 for a standard 80% train / 20% test split
# random_state=42 ensures you get the same split every time you run this (for reproducible results)
# stratify=y is very important here. Since your dataset is balanced, this ensures 
#            that both the training and testing sets have the same proportion of emotions.
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y)

# 3. (Optional) Check the shapes of your new datasets to confirm
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# 4. (Optional) Check the distribution in the training set
print("\nTraining set emotion distribution:")
print(y_train.value_counts(normalize=True))

X_train shape: (10819,)
y_train shape: (10819,)
X_test shape: (2705,)
y_test shape: (2705,)

Training set emotion distribution:
Emotion
anger       0.166744
neutral     0.166651
fear        0.166651
sadness     0.166651
joy         0.166651
surprise    0.166651
Name: proportion, dtype: float64


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Initialize the TfidfVectorizer
# We can set max_features to limit the vocabulary to the top N most frequent words,
# which helps control the dimensionality and remove very rare words.
# 5000 is a good starting point.
tfidf_vectorizer = TfidfVectorizer(max_features=5000)


# 2. Fit the vectorizer on the training data and transform it
# This learns the vocabulary (the "fit") and converts X_train to a TF-IDF matrix (the "transform")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)


# 3. Transform the test data using the *same* fitted vectorizer
# This only does the "transform" step, using the vocabulary learned from X_train.
# This is crucial to prevent data leakage.
X_test_tfidf = tfidf_vectorizer.transform(X_test)


# 4. (Optional) Check the new shapes
# The second dimension (the 5000) should match.
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_test_tfidf shape:", X_test_tfidf.shape)

X_train_tfidf shape: (10819, 5000)
X_test_tfidf shape: (2705, 5000)


In [25]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import time

# 1. Initialize the SVC model with a polynomial kernel
# kernel='poly': Specifies the polynomial kernel
# degree=3: A common starting point for the polynomial degree
# C=1.0: The regularization parameter.
print("Initializing SVC model with polynomial kernel...")
svc_poly = SVC(kernel='linear', degree=3, C=1.0)


# 2. Train the model
# This will take a moment, as SVC can be computationally intensive
print("Training the model...")
start_time = time.time() # Start timer

svc_poly.fit(X_train_tfidf, y_train)

end_time = time.time() # End timer
print(f"Training completed in {end_time - start_time:.2f} seconds.")


# 3. Make predictions on the test set
print("\nMaking predictions on the test set...")
y_pred = svc_poly.predict(X_test_tfidf)


# 4. Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy * 100:.2f}%")

print("\nClassification Report:")
# This report gives you precision, recall, and f1-score for each emotion
print(classification_report(y_test, y_pred, target_names=emotion['Emotion'].unique()))

Initializing SVC model with polynomial kernel...
Training the model...
Training completed in 5.07 seconds.

Making predictions on the test set...

Model Accuracy: 57.52%

Classification Report:
              precision    recall  f1-score   support

     sadness       0.58      0.55      0.56       450
       anger       0.68      0.57      0.62       451
    surprise       0.50      0.54      0.52       451
     neutral       0.62      0.80      0.70       451
        fear       0.53      0.50      0.51       451
         joy       0.55      0.49      0.52       451

    accuracy                           0.58      2705
   macro avg       0.58      0.58      0.57      2705
weighted avg       0.58      0.58      0.57      2705



In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC # Make sure to import SVC
import numpy as np
import time

# 1. Re-initialize the *exact same* model you used before.
#    We create a fresh, untrained model for the cross-validation process.
#    Based on your last cell, you used SVC(kernel='linear').
model_for_cv = SVC(kernel='linear', C=1.0)

# 2. Get the full training dataset
#    We use the complete X_train_tfidf and y_train.
#    cross_val_score will handle splitting it into 5 folds automatically.
X_train_full_tfidf = X_train_tfidf
y_train_full = y_train

print("Starting 5-fold cross-validation...")
print("This will train and test the model 5 times, so it may take a few moments...")
start_time = time.time()

# 3. Run the cross-validation
#    cv=5 tells it to perform 5-fold cross-validation.
#    'accuracy' is the metric we want to score.
#    n_jobs=-1 (optional) speeds up the process by using all available CPU cores.
scores = cross_val_score(model_for_cv, 
                         X_train_full_tfidf, 
                         y_train_full, 
                         cv=5, 
                         scoring='accuracy',
                         n_jobs=-1) # Use -1 to use all processors

end_time = time.time()
print(f"Cross-validation completed in {end_time - start_time:.2f} seconds.")

# 4. Print the results
print("\n--- Cross-Validation Results ---")
print(f"Accuracy for each of the 5 folds:")
for i, score in enumerate(scores):
    print(f"  Fold {i+1}: {score * 100:.2f}%")

print("\nAverage Accuracy:")
print(f"  {np.mean(scores) * 100:.2f}%")

print("\nStandard Deviation:")
print(f"  {np.std(scores) * 100:.2f}%")
print("---------------------------------")
print("\n(This average accuracy is a more reliable estimate of your model's performance.)")

Starting 5-fold cross-validation...
This will train and test the model 5 times, so it may take a few moments...
Cross-validation completed in 8.22 seconds.

--- Cross-Validation Results ---
Accuracy for each of the 5 folds:
  Fold 1: 57.07%
  Fold 2: 56.70%
  Fold 3: 57.53%
  Fold 4: 57.16%
  Fold 5: 57.37%

Average Accuracy:
  57.17%

Standard Deviation:
  0.28%
---------------------------------

(This average accuracy is a more reliable estimate of your model's performance.)
